# Generate Local Datasets

In [2]:
%pip install matplotlib tensorflow pillow TA-Lib

  Using cached TA-Lib-0.4.25.tar.gz (271 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
Failed to build TA-Lib
Note: you may need to restart the kernel to use updated packages.


  ERROR: Command errored out with exit status 1:
   command: 'C:\ProgramData\Anaconda3\python.exe' 'C:\ProgramData\Anaconda3\lib\site-packages\pip\_vendor\pep517\in_process\_in_process.py' build_wheel 'C:\Users\julia\AppData\Local\Temp\tmpet_1bd_0'
       cwd: C:\Users\julia\AppData\Local\Temp\pip-install-8p9orhyp\ta-lib_05ac6180ed684d0a81b0643d840e23a9
  Complete output (26 lines):
  <string>:77: UserWarning: Cannot find ta-lib library, installation may fail.
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-39
  creating build\lib.win-amd64-cpython-39\talib
  copying talib\abstract.py -> build\lib.win-amd64-cpython-39\talib
  copying talib\deprecated.py -> build\lib.win-amd64-cpython-39\talib
  copying talib\stream.py -> build\lib.win-amd64-cpython-39\talib
  copying talib\test_abstract.py -> build\lib.win-amd64-cpython-39\talib
  copying talib\test_data.py -> build\lib.win-amd64-cpython-39\talib
  copying talib\test_func

In [1]:
import algoseek_connector
import clickhouse_driver
import os
import logging
import pyarrow as pa
import pyarrow.dataset as ds
import pyarrow.parquet as pq
import pandas as pd
import time
import datetime

In [2]:
# create logger
logger = logging.getLogger('JupyterSamples')
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
os.mkdir('logs/GenerateDatasets')
today = time.strftime("%Y%m%d")
ch = logging.FileHandler(f"logs/GenerateDatasets/downloads_{today}.log")
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

In [3]:
# Set Credentials
DATABASE_HOST = '54.164.83.119'
DATABASE_USER = "angel_mv_hill_gmail_com"
DATABASE_PASSWORD = "UbxaVAQ572A"
client = clickhouse_driver.Client(
    DATABASE_HOST,
    user=DATABASE_USER,
    password=DATABASE_PASSWORD,
    secure=False
)
session = algoseek_connector.Session(DATABASE_HOST,DATABASE_USER,DATABASE_PASSWORD)
session.ping()
resource = algoseek_connector.DataResource(session)

## Reference Data

In [ ]:
basic_adj = resource.datagroups.USEquityReferenceData.datasets.BasicAdjustments
detailed_adj = resource.datagroups.USEquityReferenceData.datasets.DetailedAdjustments
lookup_base = resource.datagroups.USEquityReferenceData.datasets.LookupBase
sec_master_base = resource.datagroups.USEquityReferenceData.datasets.SecMasterBase

In [ ]:
basic_df = basic_adj.select(basic_adj.SecId,
                 basic_adj.Ticker,
                 basic_adj.EffectiveDate,
                 basic_adj.AdjustmentFactor,
                 basic_adj.AdjustmentReason,
                 basic_adj.EventId
).fetch()
basic_df.to_parquet('data/us_equity/reference/basic_adj.parquet')
# Detailed Adj Factors
det_df = detailed_adj.select(detailed_adj.SecId,
                    detailed_adj.Ticker,
                    detailed_adj.Name,
                    detailed_adj.ISIN,
                    detailed_adj.EffectiveDate,
                    detailed_adj.ReportDate,
                    detailed_adj.Revision,
                    detailed_adj.AdjustmentFactor,
                    detailed_adj.AdjustmentReason,
                    detailed_adj.EventType,
                    detailed_adj.EventId,
                    detailed_adj.DivPayrate,
                    detailed_adj.DivCurrency,
                    detailed_adj.DivPaymentType,
                    detailed_adj.Detail
                    ).fetch()
det_df.to_parquet('data/us_equity/reference/detailed_adj.parquet')
# Lookup Base
look_df = lookup_base.select(lookup_base.SecId,
                   lookup_base.Ticker,
                   lookup_base.StartDate,
                   lookup_base.EndDate
).fetch()
look_df.to_parquet('data/us_equity/reference/lookup_base.parquet')
# SEC Master Base
sec_df = sec_master_base.select(sec_master_base.SecId,
                       sec_master_base.ListStatus,
                       sec_master_base.SecurityDescription,
                       sec_master_base.Sic,
                       sec_master_base.Sector,
                       sec_master_base.Industry,
                       sec_master_base.SEDOL,
                       sec_master_base.Ticker,
                       sec_master_base.TickerStartToEndDate,
                       sec_master_base.Name,
                       sec_master_base.NameStartToEndDate,
                       sec_master_base.ISIN,
                       sec_master_base.ISINStartToEndDate,
                       sec_master_base.USIdentifier,
                       sec_master_base.USIdentifierStartToEndDate,
                       sec_master_base.PrimaryExchange,
                       sec_master_base.PrimaryExchangeStartToEndDate,
                       sec_master_base.FIGI
).fetch()
sec_df.to_parquet('data/us_equity/reference/sec_master.parquet')


In [ ]:
tickers = look_df['Ticker'].unique()

## Daily Dataset

In [ ]:
import asyncio
import aiohttp
import threading, os
import time
import pandas as pd
from pandas.core.frame import DataFrame
import clickhouse_driver
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.dataset as ds

num_tasks = 5
records = 50
years=[2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

results = []
failed_requests = []
Ids = set()

async def task(task_id, work_queue) -> None:
    while not work_queue.empty():
        client = clickhouse_driver.Client(
        DATABASE_HOST,
        user=DATABASE_USER,
        password=DATABASE_PASSWORD,
        secure=False
        )
        session = algoseek_connector.Session(DATABASE_HOST,DATABASE_USER,DATABASE_PASSWORD)
        resource = algoseek_connector.DataResource(session)

        Ids.add(f'Task: {task_id}, PID: {os.getpid()}, TID: {threading.get_ident()}')
        company, year = await work_queue.get()
        #endpoint = f'/inventory/{category}?id={id}'
        resource = algoseek_connector.DataResource(session)
        primary_adj = resource.datagroups.USEquityMarketData.datasets.PrimaryAdjustedOHLCDaily
        try:
            data = primary_adj.select(primary_adj.TradeDate, primary_adj.SecId, primary_adj.Ticker, primary_adj.Name, primary_adj.PrimaryExchange, primary_adj.ISIN,primary_adj.OpenTime, primary_adj.OpenPrice, primary_adj.OpenSize, primary_adj.HighTime, primary_adj.HighPrice, primary_adj.LowTime,primary_adj.LowPrice, primary_adj.CloseTime, primary_adj.ClosePrice, primary_adj.CloseSize,primary_adj.ListedMarketHoursVolume, primary_adj.ListedMarketHoursTrades,primary_adj.ListedTotalVolume, primary_adj.ListedTotalTrades, primary_adj.FinraMarketHoursVolume,primary_adj.FinraMarketHoursTrades, primary_adj.FinraTotalVolume, primary_adj.FinraTotalTrades,primary_adj.MarketVWAP, primary_adj.DailyVWAP, primary_adj.PriceAdjFactor, primary_adj.VolumeAdjFactor).filter((primary_adj.Ticker == f"{company}")&(primary_adj.TradeDate > f"{str(year)}-01-01")&(primary_adj.TradeDate <= f"{str(year+1)}-01-01")).fetch()
            data["TradeDate"] = pd.to_datetime(data["TradeDate"])
            data["Year"] = data["TradeDate"].dt.year
            data_arrow = pa.Table.from_pandas(data)

            pq.write_to_dataset(data_arrow, root_path="data/us_equity/daily/",partition_cols=["Year","SecId"], use_legacy_dataset=False)
            # data.to_parquet(f"{company}_{str(year)}.parquet")
            logger.info(f'TASK {task_id}: API request for company: {company}, year: {year}')
            work_queue.task_done()
        except:
            logger.error(f"{company}_{year} failed")

async def main() -> None:
    # Create the queue of work
    q = asyncio.Queue()

    # Put some work in the queue
    for company in tickers:
        for year in years:
            await q.put((company,year))
    # Schedule tasks concurrently
    await asyncio.gather(
        *[asyncio.create_task(task(task_id, q)) for task_id in range(num_tasks)]
    )

    # make dataframe of results
    # return pd.DataFrame(results)

def download_data():

    print('ASYNCIO')
    start_time = time.time()
    asyncio.get_event_loop().run_until_complete(main())
    # print(f'\nDataframe ({len(failed_requests)} failed requests, {len(results)} successful requests)\n {df.head()}')
    print("\n--- %s seconds ---" % (time.time() - start_time))
    print(list(Ids))

## Intraday Datasets

### TAQ Minute Bar

In [4]:
import asyncio
import aiohttp
import threading, os
import time
import pandas as pd
from pandas.core.frame import DataFrame
import clickhouse_driver
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.dataset as ds

import logging
import nest_asyncio
nest_asyncio.apply()

look = pd.read_parquet('data/us_equity/reference/lookup.parquet')
tickers = look['Ticker'].unique()

num_tasks = 10
records = 50
years=[2022]

results = []
failed_requests = []
Ids = set()

async def task(task_id, work_queue) -> None:
    while not work_queue.empty():

        DATABASE_HOST = '54.164.83.119'
        DATABASE_USER = "angel_mv_hill_gmail_com"
        DATABASE_PASSWORD = "UbxaVAQ572A"
        client = clickhouse_driver.Client(
        DATABASE_HOST,
        user=DATABASE_USER,
        password=DATABASE_PASSWORD,
        secure=False
        )
        session = algoseek_connector.Session(DATABASE_HOST,DATABASE_USER,DATABASE_PASSWORD)

        Ids.add(f'Task: {task_id}, PID: {os.getpid()}, TID: {threading.get_ident()}')
        print(f'Task: {task_id}, PID: {os.getpid()}, TID: {threading.get_ident()}')
        company, year = await work_queue.get()
        #endpoint = f'/inventory/{category}?id={id}'
        resource = algoseek_connector.DataResource(session)
        # primary_adj = resource.datagroups.USEquityMarketData.datasets.PrimaryAdjustedOHLCDaily
        taq_min = resource.datagroups.USEquityMarketData.datasets.TradeAndQuoteMinuteBar
        try:
            data = taq_min.select('TradeDate', 'BarDateTime', 'Ticker','SecId', 'OpenBarTimeOffset','OpenBidPrice', 'OpenBidSize', 'OpenAskPrice', 'OpenAskSize','FirstTradeTimeOffset', 'FirstTradePrice', 'FirstTradeSize','HighBidTimeOffset', 'HighBidPrice', 'HighBidSize', 'HighAskTimeOffset','HighAskPrice', 'HighAskSize', 'HighTradeTimeOffset', 'HighTradePrice','HighTradeSize','LowBidTimeOffset', 'LowBidPrice', 'LowBidSize','LowAskTimeOffset', 'LowAskPrice', 'LowAskSize', 'LowTradeTimeOffset','LowTradePrice', 'LowTradeSize', 'CloseBarTimeOffset', 'CloseBidPrice','CloseBidSize', 'CloseAskPrice', 'CloseAskSize', 'LastTradeTimeOffset','LastTradePrice', 'LastTradeSize', 'MinSpread', 'MaxSpread','CancelSize', 'VolumeWeightPrice', 'NBBOQuoteCount', 'TradeAtBid','TradeAtBidMid', 'TradeAtMid', 'TradeAtMidAsk', 'TradeAtAsk','TradeAtCrossOrLocked', 'Volume', 'TotalTrades', 'FinraVolume','FinraVolumeWeightPrice', 'UptickVolume', 'DowntickVolume','RepeatUptickVolume', 'RepeatDowntickVolume', 'UnknownTickVolume','TradeToMidVolWeight', 'TradeToMidVolWeightRelative', 'TimeWeightBid','TimeWeightAsk').filter(taq_min.TradeDate.between('2022-01-01','2022-12-31') & (taq_min.Ticker == f"{company}")).fetch()
            data["TradeDate"] = pd.to_datetime(data["TradeDate"])
            data['BarDateTime'] = pd.to_datetime(data["TradeDate"])
            data["Year"] = data["TradeDate"].dt.year
            data["Month"] = data["TradeDate"].dt.month

            data_arrow = pa.Table.from_pandas(data)

            pq.write_to_dataset(data_arrow, root_path="data/us_equity/intraday/",partition_cols=["Year","Month","SecId"], use_legacy_dataset=False)
            # data.to_parquet(f"{company}_{str(year)}.parquet")
            logger.info(f'TASK {task_id}: API request for company: {company}, year: {year}')
            results.append(company)
            work_queue.task_done()

        except:
            logger.error(f"{company}_{year} failed")
            failed_requests.append(company)

async def main() -> DataFrame:
    # Create the queue of work
    q = asyncio.Queue()

    # Put some work in the queue
    for company in tickers:
        for year in years:
            await q.put((company,year))

    # Schedule tasks concurrently
    await asyncio.gather(
        *[asyncio.create_task(task(task_id, q)) for task_id in range(num_tasks)]
    )

    # make dataframe of results
    return pd.DataFrame(results)


print('ASYNCIO')
start_time = time.time()
df = asyncio.get_event_loop().run_until_complete(main())
# print(f'\nDataframe ({len(failed_requests)} failed requests, {len(results)} successful requests)\n {df.head()}')
print("\n--- %s seconds ---" % (time.time() - start_time))
print(list(Ids))

ASYNCIO
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260
Task: 0, PID: 11736, TID: 40260


ExecutionError: result_bytes = 107374359922/107374182400. Interval will end at 2023-01-21 00:00:00. Name of quota template: `qD_angel_mv_hill_gmail_com`. 

In [3]:
asyncio.get_event_loop().stop()

In [7]:
with open('logs/GenerateDatasets/successful.txt','w') as fp:
    for item in results:
        fp.write("%s\n" % item)
    print('done')
results

done


['SLF',
 'SFM',
 'NTB',
 'TNP',
 'CB',
 'LOGI',
 'DB',
 'ELTK',
 'AIP',
 'AUDC',
 'BOSC',
 'BWEB',
 'ALVR',
 'CAMT',
 'CHKP',
 'CYRN',
 'CGEN',
 'ESLT',
 'WILC',
 'CRNT',
 'GILT',
 'ICL',
 'IGLD',
 'ITRN',
 'SNT',
 'MGIC',
 'SBET',
 'MNDO',
 'NVMI',
 'ELLO',
 'OBAS',
 'RADI',
 'RADA',
 'RDCM',
 'RVSN',
 'RDWR',
 'SILC',
 'SPCB',
 'TARO',
 'TATT',
 'TSEM',
 'FLWS',
 'FCCY',
 'SRCE',
 'FNHC',
 'TW',
 'ANSS',
 'COMS',
 'TDSC',
 'DDD',
 'ACMR',
 'AGE',
 'CAS',
 'AOS',
 'AAON',
 'AIR',
 'AAN',
 'VCEL',
 'ABT',
 'ABCB',
 'ANF',
 'ABMD',
 'AAC',
 'AIH',
 'ABM',
 'AKR',
 'AXDX',
 'ABEO',
 'ACET',
 'ACU',
 'ACNB',
 'ATVI',
 'EPAC',
 'ADX',
 'AE',
 'ADPT',
 'ADCT',
 'NSP',
 'ADBE',
 'TAP',
 'ADTN',
 'AEIS',
 'AMD',
 'API',
 'HRTX',
 'RCRT',
 'DENN',
 'AEHR',
 'ACY',
 'MTMT',
 'AIM',
 'AES',
 'AEMD',
 'AMG',
 'AFL',
 'AG',
 'AGCO',
 'AGIL',
 'A',
 'AGL',
 'ADC',
 'APD',
 'AIRT',
 'AKAM',
 'ALG',
 'ALK',
 'AIN',
 'ALB',
 'AA',
 'ARNC',
 'HWM',
 'ALRS',
 'ALEX',
 'MATX',
 'ALX',
 'ARE',
 'ALFA',
 '

In [8]:
with open('logs/GenerateDatasets/fails.txt','w') as fp:
    for item in failed_requests:
        fp.write("%s\n" % item)
    print('done')

done


In [9]:
len(results)


3213

In [10]:
len(failed_requests)

5107